## Este notebook es encargado utilizar el CSV con las prruebas consolidadas para crear y entrenar un modelo

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn import preprocessing
import math
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from collections import namedtuple
import sys
import pickle

In [2]:
biometrics_df = pd.read_csv("./1_standarized_biometrics.csv")
biometrics_df.head()

,HR,RR,HRV,MicroSiemens,SCR,SCR_MIN,PhaseName,ArousalMean,Aroused,TimeStamp,...,HR-2,HR-1,HRV-4,HRV-3,HRV-2,HRV-1,MicroSiemens-4,MicroSiemens-3,MicroSiemens-2,MicroSiemens-1
0,0.180957,-0.246737,-0.638383,1.604263,0,0,HA_PV,6.07,1,2020-10-25 19:03:44,...,0.180957,0.180957,-0.638383,-0.638383,-0.638383,-0.638383,1.604263,1.604263,1.604263,1.604263
1,0.463350,-0.508771,-0.930989,1.357679,0,0,HA_PV,6.07,1,2020-10-25 19:03:44,...,0.463350,0.463350,-0.930989,-0.930989,-0.930989,-0.930989,1.357679,1.357679,1.357679,1.357679
2,0.419905,-0.468458,-1.085573,1.357679,0,0,HA_PV,6.07,1,2020-10-25 19:03:45,...,0.419905,0.419905,-1.085573,-1.085573,-1.085573,-1.085573,1.357679,1.357679,1.357679,1.357679
3,0.419905,-0.468458,-1.121459,1.111095,0,0,HA_PV,7.31,1,2020-10-25 19:03:46,...,0.419905,0.419905,-1.121459,-1.121459,-1.121459,-1.121459,1.111095,1.111095,1.111095,1.111095
4,0.506795,-0.549084,-1.088333,1.111095,0,0,HA_PV,7.31,1,2020-10-25 19:03:46,...,0.419905,0.419905,-0.638383,-0.930989,-1.085573,-1.121459,1.604263,1.357679,1.357679,1.111095


In [3]:
# X = biometrics_df[['MicroSiemens', 'HR', 'HRV']].values
X = biometrics_df[['MicroSiemens', 'HR', 'HRV', 'MicroSiemens-4', 'MicroSiemens-3', 'MicroSiemens-2', 'MicroSiemens-1', 'HR-4', 'HR-3', 'HR-2', 'HR-1', 'HRV-4','HRV-3','HRV-2','HRV-1']].values
y = biometrics_df['Aroused'].values

In [4]:
X

array([[ 1.60426292,  0.18095678, -0.63838286, ..., -0.63838286,
        -0.63838286, -0.63838286],
       [ 1.35767905,  0.46335022, -0.93098869, ..., -0.93098869,
        -0.93098869, -0.93098869],
       [ 1.35767905,  0.41990507, -1.08557291, ..., -1.08557291,
        -1.08557291, -1.08557291],
       ...,
       [ 0.92410292, -1.00764835, -1.10377383, ..., -1.0792364 ,
        -1.10868131, -1.12585751],
       [ 1.11904139, -0.46657089, -1.00562414, ..., -1.10868131,
        -1.12585751, -1.10377383],
       [ 1.21651063,  0.73582348, -0.90011322, ..., -1.12585751,
        -1.10377383, -1.00562414]])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

# Models Creation

In [6]:
adaboost = "adaboost"
knn = "knn"
rf = "random-forest"
rbf = "svm-rbf"
poly = "svm-poly"

modelsToRun = [adaboost, knn, rf]
Model = namedtuple('Model',['name','clf'])
models = []

### Adaboost

In [7]:
if "adaboost" in modelsToRun:
    clf = AdaBoostClassifier(n_estimators=100, random_state=42)
    #Adaboost with RandomForest
    clf2 = AdaBoostClassifier(RandomForestClassifier(n_estimators=100), n_estimators=100)
    print("Usando adaboost")
    adaboostModel = Model("adaboost", clf)
    adaboostModel2 = Model("adaboost-rf", clf2)
    models.append(adaboostModel)
    models.append(adaboostModel2)

Usando adaboost


### KNN

In [8]:
if "knn" in modelsToRun:
    clf = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')

    # Grid Search
    clf.get_params()
    params = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
    grid_search_cv = GridSearchCV(KNeighborsClassifier(), params, n_jobs=-1, verbose=1)
    print("Usando knn")
    knnModel = Model("knn", clf)
    models.append(knnModel)
    knnGridSearchModel = Model("knn-grid-search", grid_search_cv)
    models.append(knnGridSearchModel)

Usando knn


### Random Forest

In [9]:
if "random-forest" in modelsToRun:
    clf = RandomForestClassifier(random_state=42, n_estimators=100)

    # Grid Search
    rf_clf = RandomForestClassifier(random_state=42, n_estimators=100)
    params_grid = {"max_depth": [3, None], "min_samples_split": [2, 3, 10], "min_samples_leaf": [1, 3, 10], "bootstrap": [True, False], "criterion": ['gini', 'entropy']}
    grid_search_cv = GridSearchCV(rf_clf, params_grid, n_jobs=-1, cv=5, verbose=1, scoring='accuracy', iid=False)
    print("Usando random-forest")
    rfModel = Model("random-forest", clf)
    models.append(rfModel)
    rfGridSearchModel = Model("random-forest-grid-search", grid_search_cv)
    models.append(rfGridSearchModel)

Usando random-forest


### SVM rbf

In [10]:
if "svm-rbf" in models:
    clf = svm.SVC(C=0.1, gamma=0.01, kernel='rbf')
    
    # Grid Search
    pipeline = Pipeline([('clf', svm.SVC(kernel='rbf', C=1, gamma=0.1))])
    params = {'clf__C':(0.1, 0.5, 1, 2, 5, 10, 20), 'clf__gamma':(0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1)}
    grid_search_cv = GridSearchCV(pipeline, params, n_jobs=-1, cv=3, verbose=1, scoring='accuracy')
    print(f"Usando {model}")
    
    rbfModel = Model("svm-rbf", clf)
    models.append(rfModel)
    rbfGridSearchModel = Model("svm-rbf-grid-search", grid_search_cv)
    models.append(rfGridSearchModel)

### SVM Poly

In [11]:
if "svm-poly" in models:
    clf = svm.SVC(kernel='poly', degree=3)
    
    # Grid Search
    pipeline = Pipeline([('clf', svm.SVC(kernel='poly'))])
    params = {'clf__C':(0.1, 0.5, 1, 2, 5, 10, 20), 'clf__gamma':(0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1), 'clf__degree': (3, 4, 5)}
    grid_search_cv = GridSearchCV(pipeline, params, n_jobs=-1, cv=3, verbose=1, scoring='accuracy')
    print(f"Usando {model}")
    
    polyModel = Model("svm-poly", clf)
    models.append(polyModel)
    polyGridSearchModel = Model("svm-poly-grid-search", grid_search_cv)
    models.append(polyGridSearchModel)

# Model Fit

In [12]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        print("Train Result:\n")
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))
        print("Accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))

        res = cross_val_score(clf, X_train, y_train, cv=4, scoring='accuracy')
        print("Cross Validation \t")
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        print("Test Result:\n")        
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        
        res = cross_val_score(clf, X_test, y_test, cv=4, scoring='accuracy')
        print("Cross Validation \t")
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))

In [13]:
for model in models:
    print(f"\n***********************executing:{model.name}***************************\n")
    model.clf.fit(X_train, y_train)
    print_score(model.clf, X_train, y_train, X_test, y_test, train=True)
    print("\n******************************\n")
    print_score(model.clf, X_train, y_train, X_test, y_test, train=False)
    pickle.dump(model.clf, open(f'./models/model-{model.name}', 'wb'))
    print(f'\n***********************Done with {model.name}***********************\n')


***********************executing:adaboost***************************

Train Result:

Confusion Matrix: 
 [[556 240]
 [172 902]]

Accuracy score: 0.7797

Classification Report: 
               precision    recall  f1-score   support

           0       0.76      0.70      0.73       796
           1       0.79      0.84      0.81      1074

    accuracy                           0.78      1870
   macro avg       0.78      0.77      0.77      1870
weighted avg       0.78      0.78      0.78      1870


Cross Validation 	
Average Accuracy: 	 0.5460
Accuracy SD: 		 0.0502

******************************

Test Result:

Confusion Matrix: 
 [[ 93  87]
 [ 75 213]]

accuracy score: 0.6538

Classification Report: 
               precision    recall  f1-score   support

           0       0.55      0.52      0.53       180
           1       0.71      0.74      0.72       288

    accuracy                           0.65       468
   macro avg       0.63      0.63      0.63       468
weighted avg

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.1s finished


Train Result:

Confusion Matrix: 
 [[608 188]
 [144 930]]

Accuracy score: 0.8225

Classification Report: 
               precision    recall  f1-score   support

           0       0.81      0.76      0.79       796
           1       0.83      0.87      0.85      1074

    accuracy                           0.82      1870
   macro avg       0.82      0.81      0.82      1870
weighted avg       0.82      0.82      0.82      1870


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished


Fitting 5 folds for each of 10 candidates, totalling 50 fits


...

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   10.7s finished
/Users/s.gonzalez/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   11.2s finished
/Users/s.gonzalez/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


Cross Validation 	
Average Accuracy: 	 0.5876
Accuracy SD: 		 0.1373

***********************Done with random-forest-grid-search***********************



In [14]:
sys.exit()

SystemExit: 

/Users/s.gonzalez/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
clf.fit(X_train, y_train)
grid_search_cv.fit(X_train, y_train)

# Performance Measurement

In [ ]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        print("Train Result:\n")
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))
        print("Accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))

        res = cross_val_score(clf, X_train, y_train, cv=4, scoring='accuracy')
        print("Cross Validation \t")
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        print("Test Result:\n")        
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        
        res = cross_val_score(clf, X_test, y_test, cv=4, scoring='accuracy')
        print("Cross Validation \t")
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))

## Model performance

In [ ]:
print_score(clf, X_train, y_train, X_test, y_test, train=True)
print("\n******************************\n")
print_score(clf, X_train, y_train, X_test, y_test, train=False)

## Grid Search performance

In [ ]:
y_test_pred_grid = grid_search_cv.predict(X_test)

print_score(grid_search_cv, X_train, y_train, X_test, y_test, train=True)
print("\n******************************\n")
print_score(grid_search_cv, X_train, y_train, X_test, y_test, train=False)

grid_search_cv.best_estimator_
grid_search_cv.best_estimator_.get_params()
grid_search_cv.best_params_
grid_search_cv.cv_results_

# Save model to disk

In [ ]:
import pickle
# save the model to disk
pickle.dump(grid_search_cv, open('./models/model', 'wb'))
print('Done')